# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

In [1]:
import pymorphy2
import os
from tqdm import tqdm_notebook
import json
import numpy as np
from judicial_splitter import splitter as sp
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from collections import defaultdict
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from math import log
from gensim import matutils

c:\users\masha\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Препроцессинг

In [2]:
morph = pymorphy2.MorphAnalyzer()

def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [3]:
def write_doc(text, name, i):
    '''
    Write down item's metadata as .txt file.
    text: str: metadata to be wtitten
    url: str: url of an item
    return: 
    '''
    if not os.path.exists('./avito_parsed'):
        os.makedirs('./avito_parsed')
    with open(r'./avito_parsed/%s_%d.json' %(name[:-4], i), 'w', encoding='utf-8') as f:
        json.dump(text, f, ensure_ascii=False)
    return

In [4]:
for root, dirs, files in os.walk('./avito_texts'):
    for file in tqdm_notebook(files):
        with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
            text = f.read()
            parts = sp(text, 3)
            for i, p in enumerate(parts):
                lemmas = preprocessing(p)
                write_doc(lemmas, file, i)

### Итаретор по документам

In [5]:
class IterDocs(object):
    def __init__(self, text=False, lemmas=False, tagged=False):
        self.text = text
        self.lemmas = lemmas
        self.tagged = tagged
        
    def __iter__(self):
        for root, dirs, files in os.walk('./avito_parsed'):
            for i, file in enumerate(files):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    if self.tagged is True:
                        yield TaggedDocument(words=json.load(f), tags=[i])
                    elif self.text is True: 
                        yield ' '.join(json.load(f))
                    elif self.lemmas is True:
                        yield json.load(f)

# Индексация

## Обратный индекс

In [6]:
CV = CountVectorizer()

In [113]:
def save_inverted_index_base(corpus, names) -> dict:
    """
    Create inverted index by input doc collection
    :param corpus: list: input doc collection
    :param names: list: list of names for input doc collection
    :return: inverted index
    """
    avito_vec = CV.fit_transform(corpus)
    avito_df = pd.DataFrame(avito_vec.toarray(), columns=CV.get_feature_names())
    
    index = defaultdict()
    
    for col in tqdm_notebook(avito_df):
        index[col] = [names[i] for i in list(avito_df[col][avito_df[col] > 0].index)]
    
    return index

In [114]:
# сохраняем обратный индекс
inv_base = save_inverted_index_base(IterDocs(text=True), os.listdir('./avito_parsed'))

In [116]:
def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    idf = log((N - n + 0.5) / (n + 0.5))
    score = (idf * (k1 + 1) * qf) / (qf + k1 * (1 - b + b * dl / avgdl))
        
    return score

def compute_sim(query, doc, inv_index, k1, b, avgdl, N) -> float:
    """
    Compute parameters for BM25 score and pass them to the calculation function
    :param query: str: word for which to claculate BM25
    :param doc: str: doc for which to claculate BM25
    :param inv_index: default_dict: inverted index for the collection, that includes doc
    :return: score
    """
    qf = doc.count(query)
    dl = len(doc)
    
    if query in inv_index:
        n = len(inv_index[query])
    else:
        n = 0
     
    return score_BM25(qf, dl, avgdl, k1, b, N, n)

## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [117]:
from gensim.models import Word2Vec, KeyedVectors

In [118]:
model_w2v = Word2Vec.load(r'araneum_none_fasttextskipgram_300_5_2018/araneum_none_fasttextskipgram_300_5_2018.model')

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [219]:
def get_w2v_vectors(doc):
    """
    Get doc w2v vector
    :param doc: str: doc for which to compute vector
    :return: list: cpmputed vector
    """
    all_vect = list()
    for word in doc:
        try:
            all_vect.append(model_w2v.wv[word])
        except:
            continue
            
    if len(all_vect) != 0:
        d_vect = np.mean(np.array(all_vect), axis=0)
    else:
        d_vect = [0]*300
        
    return d_vect 

def save_w2v_base(texts, idx):
    """
    Save vectors for all passed documents
    :param texts: list: documents of collection 
    :param idx: list: names of documants from collection 
    :return: list: list of vectors for input text
    """
    base = list()
    for i, doc in tqdm_notebook(enumerate(texts)):
        base.append({'id': idx[i], 'text': doc, 'vec': list(get_w2v_vectors(doc))})
    return base

In [141]:
# сохраняем w2v базу для вопросов и ответов
w2v_base = save_w2v_base(IterDocs(lemmas=True), os.listdir('./avito_parsed'))

Exception in thread Thread-33:
Traceback (most recent call last):
  File "c:\users\masha\appdata\local\programs\python\python35\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "c:\users\masha\appdata\local\programs\python\python35\lib\site-packages\tqdm\_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "c:\users\masha\appdata\local\programs\python\python35\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [7]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from time import time

In [8]:
def train_doc2vec(corpus, names):
    '''
    Train custome d2v model
    :param names: pathes of the train data docs
    :return: d2v model
    '''
    
    model_d2v = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, min_alpha=0.025, epochs=200, workers=4, dm=1, seed=42)
    %time model_d2v.build_vocab(corpus)
    print(len(model_d2v.wv.vocab))
    
    %time model_d2v.train(IterDocs(tagged=True), total_examples=model_d2v.corpus_count, epochs=model_d2v.epochs, report_delay=60)
    return model_d2v

In [9]:
model_d2v = train_doc2vec(IterDocs(tagged=True), os.listdir('./avito_parsed'))

Wall time: 4.85 s
7790
Wall time: 29min 34s


In [12]:
fname = get_tmpfile("doc2vec_model_avito.model")
model_d2v.save("doc2vec_model_avito.model")

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [217]:
def get_d2v_vectors(words):
    '''
    Compute d2v vector for doc
    :param words: list: lemmas of the doc for which to compute d2v vector
    :return: list: d2v vector
    '''
    vec = model_d2v.infer_vector(words)
    return vec 

def save_d2v_base(corpus, idx):
    """
    Save d2v vectors for all passed documents
    :param texts: list: documents of collection 
    :param idx: list: names of documants from collection 
    :return: list: list of d2v vectors for input text
    """
    base = list()
    for i, doc in tqdm_notebook(enumerate(corpus)):
        base.append({'id': idx[i], 'text': doc, 'vect': list(get_d2v_vectors(doc))})
    return base

In [218]:
# сохраняем d2v базу 
d2v_base = save_d2v_base(IterDocs(lemmas=True), os.listdir('./avito_parsed'))

In [247]:
import pickle

In [255]:
with open('inv_base.pkl', 'wb') as f:
    pickle.dump(inv_base, f)

In [253]:
with open('d2v_base.pkl', 'wb') as f:
    pickle.dump(d2v_base, f)

In [254]:
with open('w2v_base.pkl', 'wb') as f:
    pickle.dump(w2v_base, f)

# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

Функция измерения близости между векторами нам пригодится:

In [164]:
def similarity(v1, v2):
    '''
    Compute cosine similarity for 2 vectors
    :param v1, v2: list: vectors
    :return: float: vectors' similarity
    '''
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    sim = np.dot(v1_norm, v2_norm)
    if sim is not None:
        return sim
    else:
        return 0

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [207]:
def search_inv(query, corpus, inv_index) -> list:
    """
    Search documents relative to query using inverted index algorithm.
    :param query: str: input text
    :param questions: list: all questions from corpus
    :param answers: list: all answers from corpus
    :param inv_index: list: questions inverted index
    :return: list: 5 relevant answers
    """
    k1 = 2.0
    b = 0.75
    file_lens = [len(file) for file in IterDocs(lemmas=True)]
    avgdl = np.mean(file_lens)
    N = len(file_lens)

    
    query_list = preprocessing(query)
    scores = list()
    
    for i, doc in enumerate(corpus):
        score = 0
        for word in query_list:
            score += compute_sim(word, doc, inv_index, k1, b, avgdl, N)
        scores.append([i, score])
        
    ranked = sorted(scores, key = lambda x: x[1], reverse=True)
    
    result = list()
    names = list()
    i = 0
    while len(result) < 5:
        doc = ranked[i]
        name = os.listdir('./avito_parsed')[doc[0]][:-7]
        if name[-1] is '_':
            name = name[:-1]
        name += '.txt'

        if not name in names:
            names.append(name)            
            with open('./avito_texts/%s' %(name), 'r', encoding='utf-8') as f:
                result.append(f.read())
        i += 1

    return result

def search_w2v(query, w2v_base) -> list:
    """
    Search documents relative to query using inverted w2v algorithm.
    :param query: str: input text
    :param w2v_base_quest: list: all questions' vectors from corpus
    :param answers: list: all answers from corpus
    :return: list: 5 relative answers
    """
    
    similarities = list()

    for part in sp(query, 3):
        lemmas = preprocessing(query)
        vec = get_w2v_vectors(lemmas)
    
        for doc in w2v_base:
            s = similarity(vec, doc['vec'])
            similarities.append({'id': doc['id'], 'sim': s})

    ranked = sorted(similarities, key=lambda x: x['sim'], reverse=True)
    
    result = list()
    names = list()
    i = 0
    while len(result) < 5:
        doc = ranked[i]
        name = doc['id'][:-7]
        if name[-1] is '_':
            name = name[:-1]
        name += '.txt'

        if not name in names:
            names.append(name)            
            with open('./avito_texts/%s' %(name), 'r', encoding='utf-8') as f:
                result.append(f.read())
        i += 1
    
    return result
    
def search_d2v(query, d2v_base) -> list:
    """
    Search documents relative to query using inverted d2v algorithm.
    :param query: str: input text
    :param d2v_base_quest: list: all questions' vectors from corpus
    :param answers: list: all answers from corpus
    :return: list: 5 relative answers
    """
    similarities = list()

    for part in sp(query, 3):
        lemmas = preprocessing(query)
        vec = get_d2v_vectors(lemmas)
    
        for doc in d2v_base:
            s = similarity(vec, doc['vect'])
            similarities.append({'id': doc['id'], 'sim': s})

    ranked = sorted(similarities, key=lambda x: x['sim'], reverse=True)
    
    result = list()
    names = list()
    i = 0
    while len(result) < 5:
        doc = ranked[i]
        name = doc['id'][:-7]
        if name[-1] is '_':
            name = name[:-1]
        name += '.txt'

        if not name in names:
            names.append(name)            
            with open('./avito_texts/%s' %(name), 'r', encoding='utf-8') as f:
                result.append(f.read())
        i += 1
    
    return result

После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

In [205]:
def search(query, search_method):
    if search_method == 'inverted_index':
        search_result = search_inv(query, IterDocs(lemmas=True), inv_base)
    elif search_method == 'word2vec':
        search_result = search_w2v(query, w2v_base)
    elif search_method == 'doc2vec':
        search_result = search_d2v(query, d2v_base)
    else:
        raise TypeError('unsupported search method')
    return search_result

In [208]:
for r in search('балет', 'doc2vec'):
    print(r)

https://www.avito.ru/moskva/bilety_i_puteshestviya/bilety_v_bolshoy_teatr_na_spektakli_1521091832
Билеты в Большой театр на спектакли купить в Москве на Avito — Объявления на сайте Авито
№ 1521091832, размещено 8 октября в 14:49
ОКТЯБРЬ:ИСТОРИЧЕСКАЯ СЦЕНАЧисло:9,10.              Жизель (балет) 12,13.            Анна Карен на (балет) 14.                 Реквием (концерт) 16,17.            Драгоценности (балет) 20,21.            Нуриев (балет) 23,24.            Травиата (опера) 25,27,28.       Спартак (балет) 26.                 Лебединое озеро (балет) 30,31.            Драгоценности (балет) НОВАЯ СЦЕНА Число:9,10,11.          Так поступают все женщины 17,19,21,23.   Альбина (опера) 18,20,21.        Путеводитель по аркестру                        (театрализованный концерт) Цены на билеты разные, все зависит от места. Звоните! Подберём лучшие места для Вас!Доставка по Москве!Если интересуют другие спектакли в Большом театре, то можете обращаться!
Москва
м. Театральная
https://www.avito.ru

In [209]:
from flask import Flask, render_template, request, redirect, url_for
import json

app = Flask(__name__)

   
@app.route('/')
def search_fucntion():
    if request.args:
        method = rquest.args['search_method']
        query = rquest.args['query']
        return render_template('results.html')
    else:
        return render_template('main.html')
    
if __name__ == '__main__':
    app.run(debug = True)


In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import json

app = Flask(__name__)

   
@app.route('/')
def search_fucntion():
    if request.args:
        method = rquest.args['search_method']
        query = rquest.args['query']
        return render_template('results.html')
    else:
        return render_template('main.html')
    
if __name__ == '__main__':
    app.run(debug = True)
